---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
doc = {
    'housing': 'City_Zhvi_AllHomes.csv',
    'towns': 'university_towns.txt',
    'gdp': 'gdplev.xls'
}

In [ ]:
def get_towns_text_file():
    data = None
    with open(doc['towns'], 'r') as f:
        lines = f.readlines()
        data = read_lines_towns_text_file(lines)
        f.close()
    return data

def read_lines_towns_text_file(lines):
    data = []
    state = None
    state_separator = '[edit]'
    region_separator = ' ('
    for line in lines:
        if line.find(state_separator) > 0:
            state = line.split(state_separator)[0]
        else:          
            data.append((state, line.split(region_separator)[0]))
    return data        
    
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = get_towns_text_file()
    df = pd.DataFrame(data, columns=['State', 'RegionName'])
    
    return df

# df = get_list_of_university_towns()
# display(df.loc[df['State'] == 'Michigan'])

In [ ]:
def get_gdp_data(init_quarter):
    df = pd.read_excel(doc['gdp'], skiprows=7, usecols=range(4, 7))
    columns = ('Quarter', 'GDP current BUSD', 'GDP chained 2009 BUSD')
    df = df.rename(columns={'Unnamed: {}'.format(i):x for i, x in enumerate(columns)})
    index_init_quarter = df.loc[df['Quarter'] == init_quarter].index.values[0]
    df = df[index_init_quarter::].reset_index().drop('index', axis=1)
    return df

def get_gdp_change(df):
    df['GDP_delta'] = df['GDP chained 2009 BUSD'].diff()
    df['GDP_delta (%)'] = df['GDP chained 2009 BUSD'].pct_change() * 100.0
    return df

def define_if_quarter_is_in_recession(df):
    '''Recession: defined as starting with two consecutive quarters of GDP decline, 
    and ending with two consecutive quarters of GDP growth'''
    x = df['GDP_delta'].values
    y = []
    in_recession = 0
    possible_recession = False
    for index, delta in enumerate(x):
        if in_recession > 0:
            if delta > 0.0 and x[index-1] > 0.0:
                in_recession += 1
            y.append(True)
        else:
            if delta >= 0.0:
                y.append(False)
                if possible_recession:
                    y.append(False)
                    possible_recession = False
            elif delta < 0.0:
                if possible_recession is True:
                    y.append(True)
                    y.append(True)
                    possible_recession = False
                    in_recession = 1
                else:
                    possible_recession = True
            elif np.isnan(delta):
                y.append(False)
        if in_recession > 1:
            in_recession = 0
    assert len(x) == len(y), '{} != {}'.format(len(x), len(y))
    df['Recession'] = y
    return df
        
def get_gdp_required_data():
    df = get_gdp_data('2000q1')
    df = get_gdp_change(df)
    df = define_if_quarter_is_in_recession(df)
    return df
    
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = get_gdp_required_data()
#     display(df[30:50])
    out = df.loc[df['Recession'] == True].sort_values('Quarter').iloc[0]['Quarter']
    return out

display(get_recession_start())

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = get_gdp_required_data()
    out = df.loc[df['Recession'] == True].sort_values('Quarter', ascending=False).iloc[0]['Quarter']
    return out

display(get_recession_end())

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = get_gdp_required_data()
    out = df.loc[df['Recession'] == True].sort_values('GDP chained 2009 BUSD').iloc[0]['Quarter']
    return out

display(get_recession_bottom())

In [ ]:
def read_housing_data():
    df = pd.read_csv(doc['housing'])
    return df

def get_quarters_with_months_given_initial_and_end_month(initial, end):
    quarters = {}
    for month in range(initial, end + 1):
        q = define_quarter(month)
        if q in quarters:
            quarters[q].append(month)
        else:
            quarters[q] = [month]
    return quarters

def define_quarter(month):
    if month <= 3:
        return 'q1'
    elif month <= 6:
        return 'q2'
    elif month <= 9:
        return 'q3'
    elif month <= 12:
        return 'q4'
    else:
        assert False, 'Invalid month: {}'.format(month)

def get_month_in_str(month):
    if month <= 9:
        return '0{}'.format(month)
    else:
        return str(month)
        

def get_data_in_year(df, year, initial_year, initial_month, end_year, end_month):
    quarters_included = []
    if year == initial_year:
        quarters = get_quarters_with_months_given_initial_and_end_month(initial_month, 12)
    elif year == end_year:
        quarters = get_quarters_with_months_given_initial_and_end_month(1, end_month)
    else:
        quarters = get_quarters_with_months_given_initial_and_end_month(1, 12)

    for quarter, months in quarters.items():
        columns = ['{}-{}'.format(year, get_month_in_str(m)) for m in months]
#         df['{}{}'.format(year, quarter)] = df.apply(lambda r: \
#                 np.mean([r[col] for col in columns]), axis=1)
        df['{}{}'.format(year, quarter)] = df[columns].mean(axis=1)
        quarters_included.append('{}{}'.format(year, quarter))
    
    return df, quarters_included
    
    
def get_mean_quarter_house_prices(df, initial_year, initial_month, end_year, end_month):
    quarters = []
    for year in range(initial_year, end_year + 1):
        df, quarters_included = get_data_in_year(df, year, initial_year, initial_month, end_year, end_month)
        quarters += quarters_included
    df = df[get_columns_of_interest() + quarters]
    return df

def get_columns_of_interest():
    return ['State', 'RegionName']
            
def set_columns_per_quarter(df, initial_year, initial_month, end_year, end_month):
    '''A quarter is a specific three month period, 
    Q1 is January through March, 
    Q2 is April through June, 
    Q3 is July through September, 
    Q4 is October through December.
    '''
    df = get_mean_quarter_house_prices(df, initial_year, initial_month, end_year, end_month)
    return df
    

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = read_housing_data()
    df = set_columns_per_quarter(df, 2000, 1, 2016, 8)
    df = df.set_index(['State', 'RegionName'])
    out = df
    assert out.shape == (10730, 67), '{} != {}'.format(out.shape, (10730, 67))
    return out

# out = convert_housing_data_to_quarters()
# display(out)

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df_town = 
    
    return "ANSWER"